In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/Users/charming/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
nb_classes = 10

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Xavier initializer

In [4]:
nb_classes = 10
X = tf.placeholder(tf.float32,shape=[None,784])
Y = tf.placeholder(tf.float32,shape=[None,nb_classes])

In [14]:
#tf.get_variable_scope().reuse_variables()
# get_variable 재사용 하고 싶을 뗴
#tf.get_variable_scope().reuse == False
# 재사용 막을 때 
# 혹은
# tf.reset_default_graph()

False

In [5]:
W1 = tf.get_variable("W1",shape=[784,512],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X,W1)+b1)

In [6]:
W2 = tf.get_variable("W2",shape=[512,256],initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1,W2)+b2)

In [7]:
W3 = tf.get_variable('W3',shape=[256,256],initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([256]))
L3 = tf.nn.relu(tf.matmul(L2,W3)+b3)

In [8]:
W4 = tf.get_variable('W4',shape=[256,10],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([10]))
hypo = tf.matmul(L3,W4) + b4

In [17]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypo,labels=Y))

In [18]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [23]:
import random

In [39]:
training_epoch = 15
batch_size = 100

for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost,optimizer], feed_dict={X:batch_xs,Y:batch_ys})
        avg_cost += c / total_batch
    print("Epoch :", epoch+1,"cost :",avg_cost)
        
# evlauation
pred = tf.equal(tf.argmax(hypo,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(pred,tf.float32))
print("ACC :", sess.run(accuracy,feed_dict={X:mnist.test.images,Y:mnist.test.labels}))

# check
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(hypo, 1), feed_dict={X: mnist.test.images[r:r + 1]}))


Epoch : 1 cost : 0.007290241019883966
Epoch : 2 cost : 0.0032537928813151885
Epoch : 3 cost : 0.003117245164716643
Epoch : 4 cost : 0.006969381122655893
Epoch : 5 cost : 0.007884268338674354
Epoch : 6 cost : 0.002014411644479941
Epoch : 7 cost : 0.00028700129294494093
Epoch : 8 cost : 0.0008374320415107474
Epoch : 9 cost : 0.010070129349702675
Epoch : 10 cost : 0.005881968104869045
Epoch : 11 cost : 0.0015720886015899715
Epoch : 12 cost : 0.005491751598087195
Epoch : 13 cost : 0.006406815786211916
Epoch : 14 cost : 0.00277883059404197
Epoch : 15 cost : 0.0016623176080959802
ACC : 0.9831
Label:  [0]
Prediction:  [0]


In [36]:
r = random.randint(0, mnist.test.num_examples - 1)
sess.run(tf.argmax(mnist.test.labels[r:r+1],1))

array([0])

In [37]:
print("Prediction: ", sess.run(
    tf.argmax(hypo, 1), feed_dict={X: mnist.test.images[r:r + 1]}))

Prediction:  [0]


# Dropout

In [40]:
keep_prob = tf.placeholder(tf.float32)

In [42]:
L1 = tf.nn.dropout(L1,keep_prob=keep_prob)
L2 = tf.nn.dropout(L2,keep_prob=keep_prob)
L3 = tf.nn.dropout(L3,keep_prob=keep_prob)
hypothesis = tf.matmul(L3,W4) + b4

In [43]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [44]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [59]:
for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size )
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c,_ = sess.run([cost,optimizer],feed_dict={X:batch_xs,Y:batch_ys,keep_prob:0.7})
        avg_cost += c / total_batch
    print("Epoch",(epoch+1),"cost:",avg_cost)

#accuracy
prediction = tf.equal(tf.argmax(hypothesis,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
print("Accuracy :", sess.run(accuracy,feed_dict={X:mnist.test.images,Y:mnist.test.labels,keep_prob:1}))

Epoch 1 cost: 0.006226872141935575
Epoch 2 cost: 0.007354147841420151
Epoch 3 cost: 0.009027456734769125
Epoch 4 cost: 0.004648973153084058
Epoch 5 cost: 0.00784125447112165
Epoch 6 cost: 0.010542024896145884
Epoch 7 cost: 0.008152259053950955
Epoch 8 cost: 0.00970441652604077
Epoch 9 cost: 0.0028878256398361243
Epoch 10 cost: 0.00750993659247709
Epoch 11 cost: 0.006178755116183051
Epoch 12 cost: 0.010134590493041603
Epoch 13 cost: 0.0076998470065785295
Epoch 14 cost: 0.005146414236694288
Epoch 15 cost: 0.005471726007065953
Accuracy : 0.9826
